<a href="https://colab.research.google.com/github/somwrks/AI-Solar-Panel/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self Rotatory Solar Panel

This Project utilizes various vision models to train on planet sun labelled datasets and perform validation checks to determine which model performs the best and converts the best model to tinyML Model to make it usable by microcontroller

## Installing Dependencies

In [7]:
%pip install tensorflow tensorflow-datasets roboflow ultralytics torch torchvision matplotlib tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.7/901.7 kB 23.1 MB/s eta 0:00:00


## Importing Libraries

In [15]:
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset
from ultralytics import YOLO
from roboflow import Roboflow
import matplotlib.pyplot as plt
from tqdm import tqdm
import yaml

import os

In [9]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Loading Datasets

In [16]:



# Load Roboflow dataset
rf = Roboflow(api_key="j1wL1N5IgwOIiKW4RYuo")
project = rf.workspace("1009727588-qq-com").project("sun-nxvfz")
roboflow_dataset = project.version(2).download("yolov5")

# Prepare dataset paths
dataset_path = roboflow_dataset.location
train_path = os.path.join(dataset_path, 'train')
valid_path = os.path.join(dataset_path, 'valid')
test_path = os.path.join(dataset_path, 'test')

# Create YAML file for YOLOv5
yaml_content = {
    'train': train_path,
    'val': valid_path,
    'test': test_path,
    'nc': len(os.listdir(os.path.join(dataset_path, 'train', 'labels'))),
    'names': [f'class_{i}' for i in range(len(os.listdir(os.path.join(dataset_path, 'train', 'labels'))))]
}

yaml_path = os.path.join(dataset_path, 'dataset.yaml')
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)



loading Roboflow workspace...
loading Roboflow project...


## Setting Up Models


### 1. Custom CNN

In [ ]:
def create_custom_cnn():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model



In [ ]:
custom_cnn = create_custom_cnn()
custom_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Custom CNN on SUN397 dataset
custom_cnn_history = custom_cnn.fit(sun397_train, epochs=10, validation_data=sun397_test)


### 2. YoloV5 (LibRT)

In [17]:
# Load a pretrained YOLOv5 model
yolo_model = YOLO('yolov5s.pt')

# Train YOLOv5 on Roboflow dataset
yolo_results = yolo_model.train(data=yaml_path, epochs=50, imgsz=640)


PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 115MB/s] 


Ultralytics 8.3.52 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov5s.pt, data=/content/sun-2/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

100%|██████████| 755k/755k [00:00<00:00, 20.9MB/s]


Overriding model.yaml nc=80 with nc=115

                   from  n    params  module                                       arguments                     
  0                  -1  1      3520  ultralytics.nn.modules.conv.Conv             [3, 32, 6, 2, 2]              
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     18816  ultralytics.nn.modules.block.C3              [64, 64, 1]                   
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    115712  ultralytics.nn.modules.block.C3              [128, 128, 2]                 
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  3    625152  ultralytics.nn.modules.block.C3              [256, 256, 3]                 
  7                  -1  1   1180672  ultralyti

...

      50/50      4.19G      1.479      1.993      1.279          3        640: 100%|██████████| 8/8 [00:03<00:00,  2.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.18it/s]

                   all         32         32      0.708      0.683      0.688      0.316



50 epochs completed in 0.091 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 18.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 18.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.52 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv5s summary (fused): 193 layers, 9,156,041 parameters, 0 gradients, 24.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]


                   all         32         32      0.779      0.594      0.688       0.32
               class_0         32         32      0.779      0.594      0.688       0.32
Speed: 0.3ms preprocess, 4.5ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train


In [19]:
# Evaluate YOLOv5
yolo_metrics = yolo_model.val()


Ultralytics 8.3.52 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv5s summary (fused): 193 layers, 9,156,041 parameters, 0 gradients, 24.1 GFLOPs


val: Scanning /content/sun-2/valid/labels.cache... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.72it/s]


                   all         32         32      0.778      0.594      0.686      0.317
               class_0         32         32      0.778      0.594      0.686      0.317
Speed: 0.3ms preprocess, 11.4ms inference, 0.0ms loss, 6.5ms postprocess per image
Results saved to runs/detect/train2


### MobileNetV2-SSD

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

mobilenetv2_ssd = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

mobilenetv2_ssd.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Train MobileNetV2-SSD on Roboflow dataset
mobilenetv2_ssd_history = mobilenetv2_ssd.fit(roboflow_train, epochs=10, validation_data=roboflow_test)


## Comparing Models

In [20]:
print("YOLOv5 mAP:", yolo_metrics.results_dict['metrics/mAP50-95(B)'])

# Evaluate Custom CNN
# custom_cnn_eval = custom_cnn.evaluate(sun397_test)
# print("Custom CNN Accuracy:", custom_cnn_eval[1])

# # Evaluate MobileNetV2-SSD
# mobilenetv2_ssd_eval = mobilenetv2_ssd.evaluate(roboflow_test)
# print("MobileNetV2-SSD Accuracy:", mobilenetv2_ssd_eval[1])


YOLOv5 mAP: 0.31689269588537294


## Testing Models

In [ ]:
# tbd

## Saving Model

In [ ]:
model.save('sun_tracker_model.h5')
